In [1]:
from ftsa.protocols.ourftsa22.client import Client
from ftsa.protocols.ourftsa22.server import Server
from ftsa.protocols.buildingblocks.JoyeLibert import TJLS

dimension = 10
inputsize = 16 
keysize = 2048
threshold = 14
nclients = 20
dropout = 0
publicparam, _ , _ = TJLS(nclients, threshold).Setup(keysize)

Client.set_scenario(dimension, inputsize, keysize, threshold, nclients, publicparam)
Server.set_scenario(dimension, inputsize, keysize, threshold, nclients, publicparam)


# Create clients and server

In [2]:
clients = {}
for i in range(nclients):
    idx = i+1
    clients[idx] = Client(idx)

server = Server()

### **Setup-Register** phase

In [3]:
# The clients 
allpks = {}
allpkc = {}
for i in range(nclients):
    user, pks, pkc = clients[i+1].setup_register()
    allpks[user] = pks
    allpkc[user] = pkc

In [4]:
# The server
allpks, allpkc = server.setup_register(allpks, allpkc)

### **Setup-KeySetup** phase

In [5]:
# The clients
allekshares = {}
for i in range(nclients):
    user, eshares = clients[i+1].setup_keysetup(allpks, allpkc)
    allekshares[user] = eshares

In [6]:
# The server 
allekshares = server.setup_keysetup(allekshares)

In [7]:
# The clients
for i in range(nclients):
    clients[i+1].setup_keysetup2(allekshares[i+1])

### **Online-Encrypt** phase

In [8]:
# The clients
allebshares = {}
allY = {}
# run only for alive users (nclients-dropout)
for i in range(nclients-dropout):
    clients[i+1].new_fl_step()
    user, eshares, Y = clients[i+1].online_encrypt()
    allebshares[user] = eshares
    allY[user] = Y


In [9]:
# The server
allebshares = server.online_encrypt(allebshares, allY)

### **Online-Construct** phase

In [10]:
# The clients
allbshares = {}
Yzeroshares = {}
for i in range(nclients-dropout):
    user, bshares, Yzeroshare = clients[i+1].online_construct(allebshares[i+1])
    allbshares[user] = bshares 
    Yzeroshares[user] = Yzeroshare

In [11]:
# The server
sumX = server.online_construct(allbshares, Yzeroshares.values())

In [12]:
# Verify the results
summ=clients[1].X
from operator import add
for i in range(1, nclients-dropout):
    summ = list(map(add, summ, clients[i+1].X))

print(summ)
print(sumX)
print("Verify: ", sumX  == summ )


[745713, 672630, 723813, 561831, 556422, 646468, 624169, 723906, 634101, 595072]
[745713, 672630, 723813, 561831, 556422, 646468, 624169, 723906, 634101, 595072]
Verify:  True
